# Transforming DataFrames and Series

In [ ]:
import pandas as pd

When we load a dataframe from a csv we can specify the columns we want to use with `usecols`

In [ ]:
columns = ["room_id", "host_id", "room_type", "neighborhood", "reviews", "overall_satisfaction",
               "accommodates", "bedrooms", "price"]

df = pd.read_csv("data/airbnb.csv", usecols=columns, index_col="room_id").sort_index()

In [ ]:
df.head()

## Remove rows and columns
To remove rows and columns we can use `.drop`

In order to drop rows and columns from a DataFrame, you can use the function [drop](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.drop.html). By default `.loc` removes rows based on the index value.

Drop has two important arguments:
* inplace: with this argument, you can chose if you want to transform the original DataFrame or if you want the drop function to return a copy of the transformed DataFrame. It"s default value is False, i.e, you don"t apply the transformation in the original DataFrame. You"ll see this argument in many functions that transform DataFrames. **This is usually not recommended**
* axis: with this argument, you chose if you want to drop rows (axis=0) or if you want to drop columns (axis=1). The default behaviour is to drop rows. You"ll se this argument in many functions that transform DataFrames.

For example, we can remove the row with index 6499

In [ ]:
df1 = df.drop(6499)
print(6499 in df.index)
df1

If we use `inplace=True` we modify the original dataframe

In [ ]:
df.drop(6499, inplace=True)
print(6499 in df.index)
df

Same with `loc` or `iloc`, we can drop multiple rows

In [ ]:
df.drop([29720, 29891])

If we use `axis=1` we remove columns (columns are the second axis on a dataframe)

In [ ]:
df = df.drop(["reviews", "price"], axis=1)
df

### Note: Reference versus Copy

When we assign a dataframe to a new variable, it is important to know that the new dataframe is just a reference to the original one. Thus, **modifying the new dataframe will modify the original and the other way around!**

In [ ]:
new_df = df
df.drop(["room_type", "neighborhood", "overall_satisfaction"], axis=1, inplace=True)
new_df.head()

we reload the data

In [ ]:
df = pd.read_csv("data/airbnb.csv", usecols=columns, index_col="room_id").sort_index()

So, if we want to modify a dataframe and make sure we arent modifying the original one, we can use `.copy()` that returns a copy.

In [ ]:
new_df = df.copy()
df.drop(["room_type", "neighborhood", "overall_satisfaction"], axis=1, inplace=True)
new_df.head()

## Math operations with Dataframes

### Multiplication

You can either use the `*` or the `multiply()` method to multiply columns or multiply columns by a number.

For example, we can calculate the weekly price for the listings.

In [ ]:
df = pd.read_csv("data/airbnb.csv", usecols=columns, index_col="room_id").sort_index()
df["price_per_week"] = df.price.multiply(7) # o df["price_per_week"] = df.price * 7
df.head()

### Division
We can use either `/` or `divide` to divide.

For example, we can calculate the number of people per bedroom:

In [ ]:
df["people_per_bedroom"] = df.accommodates.divide(df.bedrooms)
# df["people_per_bedroom"] = df.accommodates / df.bedrooms
df.head(10)

### Column operations

In [ ]:
df = pd.read_csv("data/airbnb.csv", usecols=columns, index_col="room_id").sort_index()

In [ ]:
df.columns

We can change the name of the columns simply assigning a new list to `df.columns`. For example, we can rename the columns and make them capitalized.

In [ ]:
df.columns = ['Host_id', 'Room_type', 'Neighborhood', 'Reviews',
       'Overall_satisfaction', 'Accommodates', 'Bedrooms', 'Price']

In [ ]:
df.head()

We can delete columns with `drop`, but we can also use the python method `del`.

In [ ]:
del df["Price"]

In [ ]:
df.head()

### Operations with strings

In [ ]:
df = pd.read_csv("data/airbnb.csv", usecols=columns, index_col="room_id").sort_index()

Those columns that are strings have a set of special methods, available under the `.str` name.

For example, if we want to set the neighbourhood name to lower case, we can use `.str.lower()`

In [ ]:
df.neighborhood.str.lower().head()

And if we want to replace the spaces on the neighbourhood with a `_`, we can do so with `.replace()`

In [ ]:
df.neighborhood = df.neighborhood.str.replace(" ", "_").str.lower()
df.head()

There are many string methods available, for example, we can use `.str.strip()` to remove the `/apt` at the end of the room_type.

In [ ]:
df.room_type.str.strip("/apt").head()

We can use `.str.cat` to interpolate strings. For example, if we want to set the index to `room_id_host_id` we can do so as:

In [ ]:
df = pd.read_csv("data/airbnb.csv", usecols=columns, index_col="room_id").sort_index()
df.index = df.index.astype(str)
df.host_id = df.host_id.astype(str)
df.index = df.index.str.cat(df.host_id.astype(str), sep="_")

In [ ]:
df.head()

### Rename

`.rename` allows us to rename indices, either the row index or the columns.

We can rename columns very easily this way.

In [ ]:
df.rename(columns={"neighborhood": "neighbourhood", "reviews": "number_reviews"}).head()

We can also rename the index.

In [ ]:
df_indexed_neighbourhood = df.set_index("neighborhood")

In [ ]:
df_indexed_neighbourhood.rename({"Belém": "Belem", "Santa Maria Maior": "Saint Mary"}).head()

### Replace

`replace` allows us to replace values on the data (not the indices)

For example, if we want to change the numerical values on the overall_satisfaction column to an ordinal scale we can do it:

In [ ]:
df.replace(
    {
        5:"Best",
        4: "Good",
        3: "OK",
        2: "Not so great",
        1: "Worst",
        0: "No Information"
    }).head()

By default  `replace` replaces the values in all the columns. We can replace values only on one column.

In [ ]:
df.overall_satisfaction.replace(
    {
        5:"Best",
        4: "Good",
        3: "OK",
        2: "Not so great",
        1: "Worst",
        0: "No Information"
    }).head()

### Group by

In [ ]:
df = pd.read_csv("data/airbnb.csv", usecols=columns).sort_index()

Group by allows us to group the dataframe based on its features.

More precisely, Pandas  group by applies a process called [split-apply-combine](https://pandas.pydata.org/pandas-docs/stable/groupby.html).
* split: Separates the dataframe based on the specified groups
* apply: Applies a function to each one of the groups
* combine: Combinanes the results into a new dataframe

For example, if we want to know how many listings every host has we can do:

In [ ]:
df.groupby("host_id")

`group_by` returns a `DataFrameGroupBy `, which is a special dataframe object that separates the dataframe by group.

In [ ]:
df[["room_id", "host_id"]].head()

In [ ]:
listings_by_host = df[["room_id", "host_id"]].groupby("host_id")
listings_by_host

Now we can count them with `count`

In [ ]:
listings_by_host.count()

By default, the columns we use to group become the index, if we want them to stay as columns we can use the argument `as_index = False`.

In [ ]:
properties_by_owner = df[["room_id", "host_id"]].groupby("host_id", as_index=False)
properties_by_owner.count().head()

For example, we can calculate the average listing price by room type and host.

In [ ]:
average_price = df.groupby(["host_id", "room_type"])[["price"]].mean()
average_price.head(10)

# Merge

`merge` allows us to join two dataframes based on one or two columns, similar to a SQL JOIN.

Now we are going to load an additional dataset that contains the population in every Lisbon district (taken from [here](https://www.citypopulation.de/php/portugal-lisboa.php))

In [ ]:
df = pd.read_csv("data/airbnb.csv", usecols=columns, index_col="room_id").sort_index()
populations = pd.read_csv("data/population_lisboa.csv")

In [ ]:
populations.head()

We will do a merge between the Airbnb listings and the populations.

merge works as follows:

```
pd.merge(left_dataframe,right_dataframe,join_type)
```

merge has different parameters:

`left_on` is the name of the column for the left dataframe.
`right_on` is the name of the column for the right dataframe.
`how` specifies the join type, how to make the union:
 - `full` takes all rows for both dataframes whether the values on the join columns exist on both dataframes or not.
 - `inner` takes only the rows on the dataframews where the values on the join columns exist on both dataframes. This is the default value.
 - `left` takes all rows for the left dataframe whether the values on the join columns exist on it dataframes or not.
 - `right` the opposite of `left`

In [ ]:
listings_with_pop = pd.merge(df, populations, left_on="neighborhood", right_on="name")

In [ ]:
listings_with_pop.shape

In [ ]:
listings_with_pop.head()

Let's check how merge works.

We can see the original number of neighbourhoods by doing `unique()`

In [ ]:
neighborhoud_list_all = df.neighborhood.unique()
neighborhoud_list_all

In [ ]:
len(neighborhoud_list_all)

We see there are 24 unique neighbourhoods

Let's see how many neighbourhoods we get when we do the merge

In [ ]:
neighborhoud_list_merge = listings_with_pop.neighborhood.unique()

In [ ]:
len(neighborhoud_list_merge)

We see there is a neighbourhood missing! Maybe there is one missing on the population dataset?

In [ ]:
set(neighborhoud_list_all) - set(neighborhoud_list_merge)

We see *São Vicente* doesn't exist on the population dataset. When we do a merge, we do an `inner` merge by default, and it removes those rows whose shared column/s don't match. In this example `*São Vicente*` exists on the Airbnbn `neighborhood` column but doesn't exist on the populations dataframe `name` column.

In [ ]:
listings_with_pop[listings_with_pop.neighborhood=="São Vicente"]

In this particular example, the reference dataframe is the Airbnb, and we are just adding additional data to it. So we would rather keep those listings in São Vicente even though we wont have population information for them. In this case we will do a left merge.

In [ ]:
listings_with_pop = pd.merge(df, populations, left_on="neighborhood", right_on="name", how="left")

Now we kept all the listings.

In [ ]:
listings_with_pop[listings_with_pop.neighborhood=="São Vicente"].head()

# Crosstab

`pd.crosstab` allows us to cross data and calculate how many observations fall under two groups.

In [ ]:
pd.crosstab(df.neighborhood, df.room_type)

We can use the argument `normalize` to get percentages instead of totals:
- `normalize="all"` returns total percentages (% of the total dataframe) 
- `normalize="index"` returns percentages per row
- `normalize="columns"` returns percentages per column

In [ ]:
pd.crosstab(df.neighborhood, df.room_type, normalize="all")

Now we can see the percentage of listings per neibourhood broken down by room type:

In [ ]:
pd.crosstab(df.neighborhood, df.room_type, normalize="index")

And we can see how many of each room type are on each neighbourhood

In [ ]:
pd.crosstab(df.neighborhood, df.room_type, normalize="columns")

# Pivot Table

`pivot_table` performs the same function as pivot tables in Excel, it turns rows into columns based on the values on the columns (it "pivots" the data).

this function has different arguments:

- `index`: the columns we want to turn into rows of the pivot table
- `columns`: the columns we want to turn into columns
- `values`: the columns we want to aggregate
- `aggfunc`: the aggregate function applied to the values (mean by default)

For example, if we want to calculate the average satisfaction by room_type for each neighbourhood:

In [ ]:
pd.pivot_table(df, index="neighborhood", 
                   columns="room_type", 
                   values=["overall_satisfaction"])

## Data processing

Let's assume we have the following dataframe:

In [ ]:
unprocessed_data = pd.DataFrame({
    "worked_hours": [5, 5.2, "sick", 8, 10],
    "work_date": ["2018-01-02", "208-01-02", "2018-01-04", "2018-01-05", "2018-01-06"],
    "employee": ["Manuel", "John", "Manuel", "John", "Manuel"]
})

unprocessed_data

By looking at the dataset, it is obvious that the variable `work_date` is a date, `worked_hours` is a number and `employee` is a string. However, because of errors on the data, Pandas has considered all columns as strings (object).

In [ ]:
unprocessed_data.dtypes

We can turn columns into numbers with `pd.to_numeric`

In [ ]:
pd.to_numeric(unprocessed_data.worked_hours)

By default, `to_numeric` fails if any value can't be converted. We can change the conversion strategy by specifying the error strategy (with the argument `errors`). For example, if we use `errors="coerce"` pandas will convert those values it can and will convert the rest to null values (NaN).

In [ ]:
pd.to_numeric(unprocessed_data.worked_hours, errors="coerce")

### Operations with datetimes

Same way, we can convert dates to datetime objects (a datatype specific for dates). We can use `pd.to_datetime` to do so. It will coerce invalid dates into nulls (NaT, `not a time`)

In [ ]:
pd.to_datetime(unprocessed_data.work_date, errors="coerce")

De esta forma podemos convertir el dataframe a su formato correcto.

In [ ]:
unprocessed_data["work_date"] = pd.to_datetime(unprocessed_data.work_date, errors="coerce")
unprocessed_data["worked_hours"] = pd.to_numeric(unprocessed_data.worked_hours, errors="coerce")

In [ ]:
unprocessed_data

In [ ]:
unprocessed_data.dtypes

Datetime columns have additional functionality, similarly to the `str.` methods with strings. We can access the datetime methods via the attribute `.dt`.

For example, we can find the day of week with `.dt.dayofweek`

In [ ]:
unprocessed_data.work_date.dt.dayofweek

In [ ]:
unprocessed_data.work_date.dt.weekday_name

### Operations with categories

In [ ]:
import pandas as pd
df = pd.read_csv("data/airbnb.csv", usecols=columns, index_col="room_id").sort_index()

Pandas can encode categorical variables in two ways, regular objects (strings), or **category**. 
Pandas provide us with a category dtype for categorical data:

    Easily identify and signal categorical columns for processing and other Python libraries
    Converting a string variable with a few different values to a categorical variable saves memory
    By converting to a categorical we can specify an order on the categories.


In [ ]:
df.dtypes

In [ ]:
df.memory_usage(index=True, deep=True)

For example, we can encode the room type and neighbourhood as categories.

In [ ]:
df = df.astype({"room_type": "category", "neighborhood": "category"})

df.dtypes

In [ ]:
df.describe(include='category')

We see we get additional information when describing categories, and not only that, the 2 columns now take significantly less memory space!

In [ ]:
df.memory_usage(index=True, deep=True)

category type columns have a way to easily convert their value (or level, that means the string they represent) to a number (that pandas uses to store them internally)

In [ ]:
df.neighborhood.cat.codes

Pandas also works well for ordinal variables. We can assign an order to a categorical variable. For example, we can set the variable `room_type` as an ordinal variable `(Entire Home > Private Room > Shared Room)`

In [ ]:
df.room_type

By default a categorical column is not ordered

In [ ]:
df.room_type.cat.ordered

We can assign the order by doing `.cat.set_categories` and passing the list of levels

In [ ]:
df.room_type = df.room_type.cat.set_categories(["Shared room","Private room", "Entire home/apt"], ordered=True)

In [ ]:
df.room_type.cat.ordered

Now because we know the sorting order of the categories, we can filter the ordinal column:

In [ ]:
df[df.room_type>"Shared room"]

When reading a csv, we can also specify dtypes so we can directly create category columns:

In [ ]:
df = pd.read_csv("data/airbnb.csv", usecols=columns, dtype={
                                            "room_type": "category",
                                            "neighborhood": "category"
})

In [ ]:
df.dtypes